In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
import time
from io import StringIO
import pandas as pd
from openpyxl import Workbook

fileSize = [261888642,20904490,14169117,21348301]
timestamp = "Oct 25, 4:09 pm – Oct 25, 5:09 pm"
dictName = {}
traceDATA_DataframeTable_df1 = {}
# Set up the WebDriver using the Service object
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Open the application's login page
driver.get('https://wk-taa.datadoghq.com/account/login?redirect=f')

# Submit the form (replace with the actual form's submit button or method)
driver.find_element(By.XPATH,'/html/body/div[1]/div/div[2]/div/div[2]/form/button').click()


# Wait for the user to manually complete the MFA
input("Please complete the MFA process and press Enter to continue...")
wait = WebDriverWait(driver, timeout=30)
# wait.until(lambda d : revealed.is_displayed())
# # After the user confirms, continue with the rest of the script
print("MFA completed, proceeding with the script...")

#wait.until(EC.url_changes('https://wk-taa.datadoghq.com/apm/home'))
# Acknowledge the User that User have Logged in Datadog Dashboard
print("Login successful, current URL:", driver.current_url)

# Need to Apply Validation Point here

#Navigate to Logs browser
driver.get('https://wk-taa.datadoghq.com/logs')
wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="single-page-app_layout_page__main-content"]/div/div[1]/div[2]/div[2]/div/header[1]/div/div/div/div[1]/div[1]/label')))
#Click On Search bar and Search Key output for the the search 
time.sleep(5)
driver.find_element(By.XPATH,'//*[@id="single-page-app_layout_page__main-content"]/div/div[1]/div[2]/div[2]/div/header[1]/div/div/div/div[1]/div[1]/div/div/div/div/div[1]').send_keys('"FileSize = 261888642"')

time.sleep(5)
#search For Date time
dateTime_InputWidget = driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div[2]/div/div[1]/div[1]/header/div/div[2]/div[2]/div/div[2]/div/div/div[1]/div/div/div/div/div/input')
    # To Make Widget Interactable with user command
driver.execute_script("arguments[0].removeAttribute('readonly')", dateTime_InputWidget)

dateTime_InputWidget.click()
    # To Perform Actions Associated to Date and Time Interaction
actions = ActionChains(driver)

# Perform a double key interaction, such as Ctrl + A to select all text, So that Complete data can be deleted at once
actions.key_down(Keys.CONTROL).send_keys('a').key_up(Keys.CONTROL).send_keys(Keys.DELETE).perform()

# Since Widget Id keeps on Changing with interaction so to deal with dyanamic Input widget, using JS contain property
input_widget = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//*[contains(@id, 'druids_form_input-text_')]"))
)
    # Ideally send_key should work but since its not working so moving to elementry step to perfom any action i.e., ActionChain
actions.send_keys_to_element(input_widget, timestamp).perform()
actions.send_keys(Keys.ENTER).perform()

# Date & Time Inserted.

time.sleep(3)

#Click Group by button to group the data into fields
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/header[1]/div/div/div/div[2]/div/div[1]/div/div/div[1]/div/div[1]/button').click()

time.sleep(3)

#click buttons to filter the data as count wise and in table
driver.find_element(By.XPATH, '//*[@id="single-page-app_layout_page__main-content"]/div/div[1]/div[2]/div[2]/div/header[1]/div/div/div/div[2]/div/div[1]/div/div/div[2]/div[1]/div/div/div/div/button/div').click()
actions.send_keys('otel.trace_id (@otel.trace_id)').perform()
actions.send_keys(Keys.ENTER).perform()

time.sleep(3)
driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/header[1]/div/div/div/div[2]/div/div[2]/div/div/div/div[4]/button').click()

#click buttons to filter by data
driver.find_element(By.XPATH, '//*[@id="single-page-app_layout_page__main-content"]/div/div[1]/div[2]/div[2]/div/header[1]/div/div/div/div[2]/div/div[1]/div/div/div[2]/div[2]/div[1]/div').click()
actions.send_keys('otel.trace_id (@otel.trace_id)').perform()
actions.send_keys(Keys.ENTER).perform()
time.sleep(3)

# Now Fetch the Table Data into the dataframe.
page_source = driver.page_source
soup = StringIO(page_source)
traceID_DataframeTable = pd.read_html(soup)[0]

# Fetching Trace Ids for Other File Sizes
for i,value in enumerate(fileSize[1:], start=1):
    # Clear the search query for next items
    driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/header[1]/div/div/div/div[1]/div[1]/div/div/div/div/button').click()
    time.sleep(2)
    # Search for Other File Sizes
    driver.find_element(By.XPATH,'//*[@id="single-page-app_layout_page__main-content"]/div/div[1]/div[2]/div[2]/div/header[1]/div/div/div/div[1]/div[1]/div/div/div/div/div[1]').send_keys(f'"FileSize = {value}"')
    actions.send_keys(Keys.ENTER).perform()
    time.sleep(2)
    # Now Fetch the Table Data into the dataframe.
    pageSourceData = StringIO(driver.page_source)
    df_name = f"{value}"  # Create unique DataFrame name for each loop iteration
    dictName[df_name] = pd.read_html(pageSourceData)[0]  # Create a DataFrame and store it in the dictionary
# To get All keys values from Dictionary so that it can be utilized further for APM traces
keyList = [i for i in dictName.keys()]

#Navigate to APM Traces
driver.get('https://wk-taa.datadoghq.com/apm/traces')
wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="single-page-app_layout_page__main-content"]/div/div[1]/div[2]/div[2]/div/header/div/div/div/div/div[1]/div[1]/div/div/div[1]/div/div[1]/div/div/div')))

#Setup the time frame for APM traces:
dateTime_InputWidget_APMTrace = driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div[2]/div/header/div/header/div/div[2]/div/div/div/div/div/div/div/input')
    # To Make Widget Interactable with user command
driver.execute_script("arguments[0].removeAttribute('readonly')", dateTime_InputWidget_APMTrace)

dateTime_InputWidget_APMTrace.click()
    # To Perform Actions Associated to Date and Time Interaction
actions = ActionChains(driver)

# Perform a double key interaction, such as Ctrl + A to select all text, So that Complete data can be deleted at once
actions.key_down(Keys.CONTROL).send_keys('a').key_up(Keys.CONTROL).send_keys(Keys.DELETE).perform()

# Since Widget Id keeps on Changing with interaction so to deal with dyanamic Input widget, using JS contain property
input_widget_APMTrace = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//*[contains(@id, 'druids_form_input-text_')]"))
)
    # Ideally send_key should work but since its not working so moving to elementry step to perfom any action i.e., ActionChain
actions.send_keys_to_element(input_widget_APMTrace, timestamp).perform()
actions.send_keys(Keys.ENTER).perform()

time.sleep(5)

#Time Setting Complete
# Setting up the search Query
#To Remove Existing String
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/header/div/div/div/div/div[1]/div[1]/div/div/div[1]/div/div[1]/div/div/button').click()
time.sleep(3)
list = traceID_DataframeTable['otel.trace_idotel.trace_id'].iloc[1:].tolist()
trace_strings = [f'trace_id:{item} ' for item in list if item]
result_string = 'service:wk.taa.ufm.filestorewebapi env:clientcollab-l1-cu operation_name:Microsoft.AspNetCore.server resource_name:"POST api/FileStore/Upload" ' + ' OR '.join(trace_strings)
#driver.find_element(By.XPATH,'//*[@id="single-page-app_layout_page__main-content"]/div/div[1]/div[2]/div[2]/div/header/div/div/div/div/div[1]/div[1]/div/div/div[1]/div/div[1]/div/div/div/span').send_keys(result_string)
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/header/div/div/div/div/div[1]/div[1]/div/div/div[1]/div/div[1]/div/div/div').send_keys(result_string)
actions.send_keys(Keys.ENTER).perform()
time.sleep(3)

#Click Button To Add one more Query
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/header/div/div/div/div/div[2]/div/button').click()
wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[1]/div[1]/div/div/div/div[3]')))
# To remove the existing string from 2nd Query box 
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[1]/div[1]/div/div/div/div[3]/div/div/div[1]/div/div[1]/div/div/button').click()

# Steps to Insert Query in the second search
trace_strings = [f'trace_id:{item} ' for item in list if item]
result_string = 'env:clientcollab-l1-cu @http.url:"https://zuscu-d002-b139-pip-l1-axs01.centralus.cloudapp.azure.com/filescan/scan"  ' + ' OR '.join(trace_strings)
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[1]/div[1]/div/div/div/div[3]/div/div/div[1]/div/div[1]/div/div/div').send_keys(result_string)
actions.send_keys(Keys.ENTER).perform()
time.sleep(3)

# Setting up the Query in APM Traces Completed

# Setting Data into tabulate form so that it can easily be fetched

# Click Table to get data in table form 
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[2]/div/div/div/div/div/div[4]/button').click()
time.sleep(3)
# Group Traces by Duration
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[2]/div[1]/div/div[2]/div[1]/div/div/div[1]/div/button').click()
actions.send_keys('Span Duration (@duration)').perform()
actions.send_keys(Keys.ENTER).perform()
time.sleep(2)

# To Group by URL:
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[2]/div[1]/div/div[2]/div[2]/div/div/div[1]/div/button').click()
actions.send_keys('URL (@http.url)').perform()
actions.send_keys(Keys.ENTER).perform()
time.sleep(2)

# XPath to Set Limit to 30
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[2]/div[1]/div/div[2]/div[2]/div/div/div[2]/div[1]/div/div[2]/button').click()
actions.send_keys('30').perform()
actions.send_keys(Keys.ENTER).perform()
time.sleep(2)

# Adding Another row to group by
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[2]/div[1]/div/div[2]/div[2]/div/div/div[3]/button').click()

# To Group by Trace ID:
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[2]/div[1]/div/div[2]/div[2]/div[2]/div/div[1]/div/button').click()
actions.send_keys('trace_id').perform()
actions.send_keys(Keys.ENTER).perform()
time.sleep(2)

# XPath to Set Limit to 30
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[2]/div[1]/div/div[2]/div[2]/div[2]/div/div[2]/div[1]/div/div[2]/button').click()
actions.send_keys('30').perform()
actions.send_keys(Keys.ENTER).perform()
time.sleep(2)

# Now Fetch the Table Data into the dataframe.
page_sourceForAPMPage = StringIO(driver.page_source)
traceDATA_DataframeTable = pd.read_html(page_sourceForAPMPage)[0]
traceDATA_DataframeTable.columns = ['Http.URL', 'Trace_ID', 'Duration']

#Fetch Traces for Other Trace Ids.
for i,value in enumerate(fileSize[1:], start=1):
    #Remove Existing String
    #driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/header/div/div/div/div/div[1]/div[1]/div/div/div[1]/div/div[1]/div/div/button').click()
    driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[1]/div[1]/div/div/div/div[1]/div/div/div[1]/div/div[1]/div/div/button').click()
    time.sleep(3)
    try:
        list = dictName[keyList[i]]['otel.trace_idotel.trace_id'].iloc[1:].tolist()
    except:
        list = ["NA"]
    trace_strings = [f'trace_id:{item} ' for item in list if item]
    result_string = 'service:wk.taa.ufm.filestorewebapi env:clientcollab-l1-cu operation_name:Microsoft.AspNetCore.server resource_name:"POST api/FileStore/Upload" ' + ' OR '.join(trace_strings)
    #driver.find_element(By.XPATH,'//*[@id="single-page-app_layout_page__main-content"]/div/div[1]/div[2]/div[2]/div/header/div/div/div/div/div[1]/div[1]/div/div/div[1]/div/div[1]/div/div/div/span').send_keys(result_string)
    driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[1]/div[1]/div/div/div/div[1]/div/div/div[1]/div/div[1]/div/div/div').send_keys(result_string)
    actions.send_keys(Keys.ENTER).perform()
    time.sleep(3)

    # To remove the existing string from 2nd Query box 
    driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[1]/div[1]/div/div/div/div[3]/div/div/div[1]/div/div[1]/div/div/button').click()
    
    # Steps to Insert Query in the second search
    trace_strings = [f'trace_id:{item} ' for item in list if item]
    result_string = 'env:clientcollab-l1-cu @http.url:"https://zuscu-d002-b139-pip-l1-axs01.centralus.cloudapp.azure.com/filescan/scan"  ' + ' OR '.join(trace_strings)
    driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[1]/div[1]/div/div/div/div[3]/div/div/div[1]/div/div[1]/div/div/div').send_keys(result_string)
    actions.send_keys(Keys.ENTER).perform()
    time.sleep(3)

    # Now Fetch the Table Data into the dataframe.
    pageSourceData = StringIO(driver.page_source)
    df_name = f"{value}"  # Create unique DataFrame name for each loop iteration
    traceDATA_DataframeTable_df1[df_name] = pd.read_html(pageSourceData)[0]  # Create a DataFrame and store it in the dictionary
    traceDATA_DataframeTable_df1[df_name].columns = ['Http.URL', 'Trace_ID', 'Duration']

#Writing data into Excel:
with pd.ExcelWriter('ResultedTraces.xlsx') as writer:
    # Write the first DataFrame to the first sheet
    traceDATA_DataframeTable.to_excel(writer, sheet_name= f'File Size = {fileSize[0]}')

    # Write the second DataFrame to the second sheet
    for i,value in enumerate(keyList[1:], start=1):
        try:
            traceDATA_DataframeTable_df1[keyList[i]].to_excel(writer, sheet_name=f'File Size = {fileSize[i]}')
        except:
            print(f'No Traces Available for File Size {keyList[i]}, However Trace ID is published in the downloaded CSV')
            dictName[keyList[i]].to_excel(writer, sheet_name=f'File Size = {fileSize[i]}')

# Close the browser
driver.quit()


Please complete the MFA process and press Enter to continue... 


MFA completed, proceeding with the script...
Login successful, current URL: https://wk-taa.datadoghq.com/apm/home


ValueError: No tables found

In [64]:
with pd.ExcelWriter('ResultedTraces.xlsx') as writer:
    # Write the first DataFrame to the first sheet
    traceDATA_DataframeTable.to_excel(writer, sheet_name= f'File Size = {fileSize[0]}')

    # Write the second DataFrame to the second sheet
    for i,value in enumerate(keyList[1:], start=1):
        try:
            traceDATA_DataframeTable_df1[keyList[i]].to_excel(writer, sheet_name=f'File Size = {fileSize[i]}')
        except:
            print(f'No Traces Available for File Size {keyList[i]}, However Trace ID is published in the downloaded CSV')
            dictName[keyList[i]].to_excel(writer, sheet_name=f'File Size = {fileSize[i]}')

In [71]:
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
input_element = driver.find_element(By.XPATH, '//*[@id="druids_form_input-text_8"]')
#input_element = driver.find_element(By.XPATH, '//*[@id="single-page-app_layout_page__main-content"]/div/div[1]/div[1]/header/div/div[2]/div[2]/div/div[2]/div/div/div[1]/div/div/div/div')
# Remove the readonly attribute using JavaScript
driver.execute_script("arguments[0].removeAttribute('readonly')", input_element)

input_element.click()

actions = ActionChains(driver)

# Perform a double key interaction, such as Ctrl + A to select all text
actions.key_down(Keys.CONTROL).send_keys('a').key_up(Keys.CONTROL).send_keys(Keys.DELETE).perform()
input_widget = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//*[contains(@id, 'druids_form_input-text_')]"))
)
actions.send_keys_to_element(input_widget, "Oct 17, 1:38 pm – Oct 17, 5:53 pm").perform()
actions.send_keys(Keys.ENTER).perform()
#input_element.click()
#input_element.send_keys("Oct 17, 1:38 pm – Oct 17, 5:53 pm")
# Use JavaScript to clear the existing value in the input field
# driver.execute_script("arguments[0].value = '';", input_element)
#input_element.clear()
# # Now enter the new value using send_keys
#input_element.send_keys("Oct 17, 1:38 pm – Oct 17, 5:53 pm")

# # Optionally, trigger a change event to ensure the web page detects the new input
# driver.execute_script("arguments[0].dispatchEvent(new Event('change'))", input_element)

input("Press Enter to close the browser...")


Press Enter to close the browser... 


''

In [139]:
from io import StringIO
import pandas as pd
#from bs4 import BeautifulSoup
page_source = driver.page_source
soup = StringIO(page_source)
traceID_DataframeTable = pd.read_html(soup)[0]
print(tables)

         otel.trace_idotel.trace_id  uniq:otel.trace_iduniq:otel.trace_id
0                               NaN                                     8
1  6595aa4a424e9407bb856d590632278a                                     1
2  77ba340a7a6cd4e7eee9233118c605ca                                     1
3  876fe674f8acaf7bb4f48a866056a6f5                                     1
4  a76d5acca515ce3d0734612bbfb55828                                     1
5  b838e196e3f95f5bf09448a8b7961735                                     1
6  ccf78f8ee1f0accfd553fe550ae93ef7                                     1
7  e88895ced921373f68c42c49bf84c9a1                                     1
8  f47ca28738810e787bd3d8ef6474d73b                                     1


In [8]:
date_time_range_input.send_keys('Oct 15, 1:52 pm – Oct 17, 1:52 pm')

In [9]:
<input class="druids_form_input-text__input" type="text" readonly="" placeholder=" " autocomplete="off" id="druids_form_input-text_8" value="Past 15 Minutes" style="text-align: left;">

In [116]:
pip install html5lib

   ---------------------------------------- 0.0/112.2 kB ? eta -:--:--
   -------------- ------------------------- 41.0/112.2 kB 1.9 MB/s eta 0:00:01
   ------------------------------------ --- 102.4/112.2 kB 1.5 MB/s eta 0:00:01
   ---------------------------------------- 112.2/112.2 kB 1.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [118]:
driver.get('https://wk-taa.datadoghq.com/apm/traces')




In [152]:

#To Remove Existing String
driver.find_element(By.XPATH,'//*[@id="single-page-app_layout_page__main-content"]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[1]/div[1]/div/div/div/div/div/div/div[1]/div/div[1]/div/div/button').click()
list = tables['otel.trace_idotel.trace_id'].iloc[1:].tolist()
trace_strings = [f'trace_id:{item} ' for item in list if item]
result_string = 'service:wk.taa.ufm.filestorewebapi env:clientcollab-l1-cu operation_name:Microsoft.AspNetCore.server resource_name:"POST api/FileStore/Upload" ' + ' OR '.join(trace_strings)
#driver.find_element(By.XPATH,'//*[@id="single-page-app_layout_page__main-content"]/div/div[1]/div[2]/div[2]/div/header/div/div/div/div/div[1]/div[1]/div/div/div[1]/div/div[1]/div/div/div/span').send_keys(result_string)
driver.find_element(By.XPATH,'//*[@id="single-page-app_layout_page__main-content"]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[1]/div[1]/div/div/div/div/div/div/div[1]/div/div[1]/div/div/div/span').send_keys(result_string)
actions.send_keys(Keys.ENTER).perform()
time.sleep(3)

#clear existing data
#driver.find_element(By.XPATH,'//*[@id="single-page-app_layout_page__main-content"]/div/div[1]/div[2]/div[2]/div/header/div/div/div/div/div[2]/div/button
driver.find_element(By.XPATH,'//*[@id="single-page-app_layout_page__main-content"]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[1]/div[2]/div[1]/button').click()

wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="single-page-app_layout_page__main-content"]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[1]/div[1]/div/div/div/div[3]/div/div/div[1]/div/div[1]/div/div/div')))

driver.find_element(By.XPATH,'//*[@id="single-page-app_layout_page__main-content"]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[1]/div[1]/div/div/div/div[3]/div/div/div[1]/div/div[1]/div/div/button').click()

# Clear Existing data
#driver.find_element(By.XPATH,'//*[@id="single-page-app_layout_page__main-content"]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[1]/div[1]/div/div/div/div[3]/div/div/div[1]/div/div[1]/div/div/button').click()
trace_strings = [f'trace_id:{item} ' for item in list if item]
result_string = 'env:clientcollab-l1-cu @http.url:"https://zuscu-d002-b139-pip-l1-axs01.centralus.cloudapp.azure.com/filescan/scan"  ' + ' OR '.join(trace_strings)
driver.find_element(By.XPATH,'//*[@id="single-page-app_layout_page__main-content"]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[1]/div[1]/div/div/div/div[3]/div/div/div[1]/div/div[1]/div/div/div/span[1]').send_keys(result_string)
actions.send_keys(Keys.ENTER).perform()
time.sleep(3)


ElementClickInterceptedException: Message: element click intercepted: Element <button data-dd-action-name="Action" class="druids_form_action druids_form_action--is-basic druids_search-bar_clear-button druids_search-bar_rich-search-bar__clear-button" type="button" aria-label="Clear search query" title="Clear">...</button> is not clickable at point (1536, 17). Other element would receive the click: <input class="druids_form_input-text__input" type="text" placeholder=" " autocomplete="off" id="druids_form_input-text_21" value="Oct 17, 1:38 pm – Oct 17, 5:53 pm" style="text-align: left;">
  (Session info: chrome=130.0.6723.59)
Stacktrace:
	GetHandleVerifier [0x010274D3+23139]
	(No symbol) [0x00FAFCA4]
	(No symbol) [0x00E8C2A3]
	(No symbol) [0x00ED3B07]
	(No symbol) [0x00ED1F69]
	(No symbol) [0x00ECFE44]
	(No symbol) [0x00ECF43E]
	(No symbol) [0x00EC413D]
	(No symbol) [0x00EEFE7C]
	(No symbol) [0x00EC3BF4]
	(No symbol) [0x00EF0114]
	(No symbol) [0x00F09564]
	(No symbol) [0x00EEFC16]
	(No sy

In [130]:
list = [1,2,3,4,5]
trace_strings = [f'trace_id:{item} ' for item in list if item]
result_string = 'service:wk.taa.ufm.filestorewebapi env:clientcollab-l1-cu operation_name:Microsoft.AspNetCore.server resource_name:"POST api/FileStore/Upload" ' + ' OR '.join(trace_strings)
#keyString = f'{service:wk.taa.ufm.filestorewebapi env:clientcollab-l1-cu operation_name:Microsoft.AspNetCore.server resource_name:"POST api/FileStore/Upload" trace_id:{id_1} OR }'.join
result_string

'service:wk.taa.ufm.filestorewebapi env:clientcollab-l1-cu operation_name:Microsoft.AspNetCore.server resource_name:"POST api/FileStore/Upload" trace_id:1  OR trace_id:2  OR trace_id:3  OR trace_id:4  OR trace_id:5 '

In [189]:

driver.find_element(By.XPATH,'//*[@id="druids_form_field38"]/div[4]/button').click()
time.sleep(5)
# Default element of Avg
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[2]/div[1]/div/div[2]/div[1]/div/div/div[1]/div/button').click()
actions.send_keys('Span Duration (@duration)').perform()
actions.send_keys(Keys.ENTER).perform()
time.sleep(2)

# First XPath Where URL to be selected URL (@http.url)
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[2]/div[1]/div/div[2]/div[2]/div/div/div[1]/div/button').click()
actions.send_keys('URL (@http.url)').perform()
actions.send_keys(Keys.ENTER).perform()
time.sleep(2)

# XPath to Set Limit to 30
driver.find_element(By.XPATH,'//*[@id="single-page-app_layout_page__main-content"]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[2]/div[1]/div/div[2]/div[2]/div/div/div[2]/div[1]/div/div[2]/button').click()
actions.send_keys('30)').perform()
actions.send_keys(Keys.ENTER).perform()
time.sleep(2)



NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[2]/div[1]/div/div[2]/div[1]/div/div/div[1]/div/button"}
  (Session info: chrome=130.0.6723.59); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x010274D3+23139]
	(No symbol) [0x00FAFCA4]
	(No symbol) [0x00E8C2A3]
	(No symbol) [0x00ECDC66]
	(No symbol) [0x00ECDEAB]
	(No symbol) [0x00F0B992]
	(No symbol) [0x00EEFEC4]
	(No symbol) [0x00F09564]
	(No symbol) [0x00EEFC16]
	(No symbol) [0x00EC216C]
	(No symbol) [0x00EC30FD]
	GetHandleVerifier [0x012BBAA3+2727987]
	GetHandleVerifier [0x0131476E+3091710]
	GetHandleVerifier [0x0130CF52+3060962]
	GetHandleVerifier [0x010C1460+653808]
	(No symbol) [0x00FB8E5D]
	(No symbol) [0x00FB5A48]
	(No symbol) [0x00FB5BE0]
	(No symbol) [0x00FA8110]
	Bas

In [193]:
dateTime_InputWidget = driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div[2]/div/div[1]/div[1]/header/div/div[2]/div[2]/div/div[2]/div/div/div[1]/div/div/div/div/div/input')
    # To Make Widget Interactable with user command
driver.execute_script("arguments[0].removeAttribute('readonly')", dateTime_InputWidget)

dateTime_InputWidget.click()
    # To Perform Actions Associated to Date and Time Interaction
actions = ActionChains(driver)

# Perform a double key interaction, such as Ctrl + A to select all text, So that Complete data can be deleted at once
actions.key_down(Keys.CONTROL).send_keys('a').key_up(Keys.CONTROL).send_keys(Keys.DELETE).perform()

# Since Widget Id keeps on Changing with interaction so to deal with dyanamic Input widget, using JS contain property
input_widget = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//*[contains(@id, 'druids_form_input-text_')]"))
)
    # Ideally send_key should work but since its not working so moving to elementry step to perfom any action i.e., ActionChain
actions.send_keys_to_element(input_widget, "Oct 17, 1:38 pm – Oct 17, 5:53 pm").perform()
actions.send_keys(Keys.ENTER).perform()

# Date & Time Inserted.

time.sleep(5)

#Click Group by button to group the data into fields
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/header[1]/div/div/div/div[2]/div/div[1]/div/div/div[1]/div/div[1]/button').click()

time.sleep(5)

#click buttons to filter the data as count wise and in table
driver.find_element(By.XPATH, '//*[@id="single-page-app_layout_page__main-content"]/div/div[1]/div[2]/div[2]/div/header[1]/div/div/div/div[2]/div/div[1]/div/div/div[2]/div[1]/div/div/div/div/button/div').click()
actions.send_keys('otel.trace_id (@otel.trace_id)').perform()
actions.send_keys(Keys.ENTER).perform()

time.sleep(5)
driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/header[1]/div/div/div/div[2]/div/div[2]/div/div/div/div[4]/button').click()

#click buttons to filter by data
driver.find_element(By.XPATH, '//*[@id="single-page-app_layout_page__main-content"]/div/div[1]/div[2]/div[2]/div/header[1]/div/div/div/div[2]/div/div[1]/div/div/div[2]/div[2]/div[1]/div').click()
actions.send_keys('otel.trace_id (@otel.trace_id)').perform()
actions.send_keys(Keys.ENTER).perform()
time.sleep(5)

# Now Fetch the Table Data into the dataframe.
page_source = driver.page_source
soup = StringIO(page_source)
traceID_DataframeTable = pd.read_html(soup)[0]

#Navigate to APM Traces
driver.get('https://wk-taa.datadoghq.com/apm/traces')
wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="single-page-app_layout_page__main-content"]/div/div[1]/div[2]/div[2]/div/header/div/div/div/div/div[1]/div[1]/div/div/div[1]/div/div[1]/div/div/div')))

#Setup the time frame for APM traces:
dateTime_InputWidget_APMTrace = driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div[2]/div/header/div/header/div/div[2]/div/div/div/div/div/div/div/input')
    # To Make Widget Interactable with user command
driver.execute_script("arguments[0].removeAttribute('readonly')", dateTime_InputWidget_APMTrace)

dateTime_InputWidget_APMTrace.click()
    # To Perform Actions Associated to Date and Time Interaction
actions = ActionChains(driver)

# Perform a double key interaction, such as Ctrl + A to select all text, So that Complete data can be deleted at once
actions.key_down(Keys.CONTROL).send_keys('a').key_up(Keys.CONTROL).send_keys(Keys.DELETE).perform()

# Since Widget Id keeps on Changing with interaction so to deal with dyanamic Input widget, using JS contain property
input_widget_APMTrace = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//*[contains(@id, 'druids_form_input-text_')]"))
)
    # Ideally send_key should work but since its not working so moving to elementry step to perfom any action i.e., ActionChain
actions.send_keys_to_element(input_widget_APMTrace, "Oct 17, 1:38 pm – Oct 17, 5:53 pm").perform()
actions.send_keys(Keys.ENTER).perform()

time.sleep(5)

#Time Setting Complete
# Setting up the search Query
#To Remove Existing String
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[1]/div[1]/div/div/div/div/div/div/div[1]/div/div[1]/div/div/button').click()
time.sleep(3)
list = tables['otel.trace_idotel.trace_id'].iloc[1:].tolist()
trace_strings = [f'trace_id:{item} ' for item in list if item]
result_string = 'service:wk.taa.ufm.filestorewebapi env:clientcollab-l1-cu operation_name:Microsoft.AspNetCore.server resource_name:"POST api/FileStore/Upload" ' + ' OR '.join(trace_strings)
#driver.find_element(By.XPATH,'//*[@id="single-page-app_layout_page__main-content"]/div/div[1]/div[2]/div[2]/div/header/div/div/div/div/div[1]/div[1]/div/div/div[1]/div/div[1]/div/div/div/span').send_keys(result_string)
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[1]/div[1]/div/div/div/div/div/div/div[1]/div/div[1]/div/div/div').send_keys(result_string)
actions.send_keys(Keys.ENTER).perform()
time.sleep(3)

#Click Button To Add one more Query
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[1]/div[2]/div[1]/button').click()
wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[1]/div[1]/div/div/div/div[3]')))
# To remove the existing string from 2nd Query box 
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[1]/div[1]/div/div/div/div[3]/div/div/div[1]/div/div[1]/div/div/button').click()

# Steps to Insert Query in the second search
trace_strings = [f'trace_id:{item} ' for item in list if item]
result_string = 'env:clientcollab-l1-cu @http.url:"https://zuscu-d002-b139-pip-l1-axs01.centralus.cloudapp.azure.com/filescan/scan"  ' + ' OR '.join(trace_strings)
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[1]/div[1]/div/div/div/div[3]/div/div/div[1]/div/div[1]/div/div/div').send_keys(result_string)
actions.send_keys(Keys.ENTER).perform()
time.sleep(3)

# Setting up the Query in APM Traces Completed

# Setting Data into tabulate form so that it can easily be fetched

# Click Table to get data in table form 
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[2]/div/div/div/div/div/div[4]/button').click()
time.sleep(3)
# Group Traces by Duration
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[2]/div[1]/div/div[2]/div[1]/div/div/div[1]/div/button').click()
actions.send_keys('Span Duration (@duration)').perform()
actions.send_keys(Keys.ENTER).perform()
time.sleep(2)

# To Group by URL:
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[2]/div[1]/div/div[2]/div[2]/div/div/div[1]/div/button').click()
actions.send_keys('URL (@http.url)').perform()
actions.send_keys(Keys.ENTER).perform()
time.sleep(2)

# XPath to Set Limit to 30
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[2]/div[1]/div/div[2]/div[2]/div/div/div[2]/div[1]/div/div[2]/button').click()
actions.send_keys('30').perform()
actions.send_keys(Keys.ENTER).perform()
time.sleep(2)

# Adding Another row to group by
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[2]/div[1]/div/div[2]/div[2]/div/div/div[3]/button').click()

# To Group by Trace ID:
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[2]/div[1]/div/div[2]/div[2]/div[2]/div/div[1]/div/button').click()
actions.send_keys('trace_id').perform()
actions.send_keys(Keys.ENTER).perform()
time.sleep(2)

# XPath to Set Limit to 30
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[2]/div[1]/div/div[2]/div[2]/div[2]/div/div[2]/div[1]/div/div[2]/button').click()
actions.send_keys('30').perform()
actions.send_keys(Keys.ENTER).perform()
time.sleep(2)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="druids_form_input-text_21"]"}
  (Session info: chrome=130.0.6723.59); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x010274D3+23139]
	(No symbol) [0x00FAFCA4]
	(No symbol) [0x00E8C2A3]
	(No symbol) [0x00ECDC66]
	(No symbol) [0x00ECDEAB]
	(No symbol) [0x00F0B992]
	(No symbol) [0x00EEFEC4]
	(No symbol) [0x00F09564]
	(No symbol) [0x00EEFC16]
	(No symbol) [0x00EC216C]
	(No symbol) [0x00EC30FD]
	GetHandleVerifier [0x012BBAA3+2727987]
	GetHandleVerifier [0x0131476E+3091710]
	GetHandleVerifier [0x0130CF52+3060962]
	GetHandleVerifier [0x010C1460+653808]
	(No symbol) [0x00FB8E5D]
	(No symbol) [0x00FB5A48]
	(No symbol) [0x00FB5BE0]
	(No symbol) [0x00FA8110]
	BaseThreadInitThunk [0x768B7BA9+25]
	RtlInitializeExceptionChain [0x7741C0CB+107]
	RtlClearBits 

In [220]:
#Setup the time frame for APM traces:
dateTime_InputWidget_APMTrace = driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div[2]/div/header/div/header/div/div[2]/div/div/div/div/div/div/div/input')
    # To Make Widget Interactable with user command
driver.execute_script("arguments[0].removeAttribute('readonly')", dateTime_InputWidget_APMTrace)

dateTime_InputWidget_APMTrace.click()
    # To Perform Actions Associated to Date and Time Interaction
actions = ActionChains(driver)

# Perform a double key interaction, such as Ctrl + A to select all text, So that Complete data can be deleted at once
actions.key_down(Keys.CONTROL).send_keys('a').key_up(Keys.CONTROL).send_keys(Keys.DELETE).perform()

# Since Widget Id keeps on Changing with interaction so to deal with dyanamic Input widget, using JS contain property
input_widget_APMTrace = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//*[contains(@id, 'druids_form_input-text_')]"))
)
    # Ideally send_key should work but since its not working so moving to elementry step to perfom any action i.e., ActionChain
actions.send_keys_to_element(input_widget_APMTrace, "Oct 17, 1:38 pm – Oct 17, 5:53 pm").perform()
actions.send_keys(Keys.ENTER).perform()

time.sleep(5)

#Time Setting Complete
# Setting up the search Query
#To Remove Existing String
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[1]/div[1]/div/div/div/div/div/div/div[1]/div/div[1]/div/div/button').click()
time.sleep(3)
list = tables['otel.trace_idotel.trace_id'].iloc[1:].tolist()
trace_strings = [f'trace_id:{item} ' for item in list if item]
result_string = 'service:wk.taa.ufm.filestorewebapi env:clientcollab-l1-cu operation_name:Microsoft.AspNetCore.server resource_name:"POST api/FileStore/Upload" ' + ' OR '.join(trace_strings)
#driver.find_element(By.XPATH,'//*[@id="single-page-app_layout_page__main-content"]/div/div[1]/div[2]/div[2]/div/header/div/div/div/div/div[1]/div[1]/div/div/div[1]/div/div[1]/div/div/div/span').send_keys(result_string)
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[1]/div[1]/div/div/div/div/div/div/div[1]/div/div[1]/div/div/div').send_keys(result_string)
actions.send_keys(Keys.ENTER).perform()
time.sleep(3)

#Click Button To Add one more Query
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[1]/div[2]/div[1]/button').click()
wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[1]/div[1]/div/div/div/div[3]')))
# To remove the existing string from 2nd Query box 
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[1]/div[1]/div/div/div/div[3]/div/div/div[1]/div/div[1]/div/div/button').click()

# Steps to Insert Query in the second search
trace_strings = [f'trace_id:{item} ' for item in list if item]
result_string = 'env:clientcollab-l1-cu @http.url:"https://zuscu-d002-b139-pip-l1-axs01.centralus.cloudapp.azure.com/filescan/scan"  ' + ' OR '.join(trace_strings)
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[1]/div[1]/div/div/div/div[3]/div/div/div[1]/div/div[1]/div/div/div').send_keys(result_string)
actions.send_keys(Keys.ENTER).perform()
time.sleep(3)

# Setting up the Query in APM Traces Completed

# Setting Data into tabulate form so that it can easily be fetched

# Click Table to get data in table form 
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[2]/div/div/div/div/div/div[4]/button').click()
time.sleep(3)
# Group Traces by Duration
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[2]/div[1]/div/div[2]/div[1]/div/div/div[1]/div/button').click()
actions.send_keys('Span Duration (@duration)').perform()
actions.send_keys(Keys.ENTER).perform()
time.sleep(2)

# To Group by URL:
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[2]/div[1]/div/div[2]/div[2]/div/div/div[1]/div/button').click()
actions.send_keys('URL (@http.url)').perform()
actions.send_keys(Keys.ENTER).perform()
time.sleep(2)

# XPath to Set Limit to 30
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[2]/div[1]/div/div[2]/div[2]/div/div/div[2]/div[1]/div/div[2]/button').click()
actions.send_keys('30').perform()
actions.send_keys(Keys.ENTER).perform()
time.sleep(2)

# Adding Another row to group by
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[2]/div[1]/div/div[2]/div[2]/div/div/div[3]/button').click()

# To Group by Trace ID:
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[2]/div[1]/div/div[2]/div[2]/div[2]/div/div[1]/div/button').click()
actions.send_keys('trace_id').perform()
actions.send_keys(Keys.ENTER).perform()
time.sleep(2)

# XPath to Set Limit to 30
driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[2]/div[1]/div/div[2]/div[2]/div[2]/div/div[2]/div[1]/div/div[2]/button').click()
actions.send_keys('30').perform()
actions.send_keys(Keys.ENTER).perform()
time.sleep(2)

# Setting Data into tabulate form so that it can easily be fetched -> Completed
# # Close the browser
# #driver.quit()


In [222]:
# Now Fetch the Table Data into the dataframe.
page_sourceForAPMPage = StringIO(driver.page_source)
traceDATA_DataframeTable = pd.read_html(page_sourceForAPMPage)[0]
traceDATA_DataframeTable.columns = ['Http.URL', 'Trace_ID', 'Duration']

traceDATA_DataframeTable

,Http.URL,Trace_ID,Duration
0,http://ltcollaborationapi.cchaxcess.com/ufmfil...,f47ca28738810e787bd3d8ef6474d73b,23.23 s
1,http://ltcollaborationapi.cchaxcess.com/ufmfil...,b838e196e3f95f5bf09448a8b7961735,21.74 s
2,http://ltcollaborationapi.cchaxcess.com/ufmfil...,e88895ced921373f68c42c49bf84c9a1,21.62 s
3,https://zuscu-d002-b139-pip-l1-axs01.centralus...,f47ca28738810e787bd3d8ef6474d73b,17.52 s
4,https://zuscu-d002-b139-pip-l1-axs01.centralus...,b838e196e3f95f5bf09448a8b7961735,16.75 s
5,https://zuscu-d002-b139-pip-l1-axs01.centralus...,e88895ced921373f68c42c49bf84c9a1,16.55 s
6,http://ltcollaborationapi.cchaxcess.com/ufmfil...,a76d5acca515ce3d0734612bbfb55828,13.16 s
7,http://ltcollaborationapi.cchaxcess.com/ufmfil...,ccf78f8ee1f0accfd553fe550ae93ef7,12.98 s
8,http://ltcollaborationapi.cchaxcess.com/ufmfil...,876fe674f8acaf7bb4f48a866056a6f5,10.36 s
9,http://ltcollaborationapi.cchaxcess.com/ufmfil...,77ba340a7a6cd4e7eee9233118c605ca,10.29 s


In [223]:
pip install openpyxl

In [55]:
traceDATA_DataframeTable_df1 = {}
#Fetch Traces for Other Trace Ids.
for i,value in enumerate(fileSize[1:], start=1):
    print(i, value)
    #Remove Existing String
    #driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/header/div/div/div/div/div[1]/div[1]/div/div/div[1]/div/div[1]/div/div/button').click()
    driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[1]/div[1]/div/div/div/div[1]/div/div/div[1]/div/div[1]/div/div/button').click()
    time.sleep(3)
    try:
        list = dictName[keyList[i]]['otel.trace_idotel.trace_id'].iloc[1:].tolist()
    except:
        list = ["NA"]
    trace_strings = [f'trace_id:{item} ' for item in list if item]
    result_string = 'service:wk.taa.ufm.filestorewebapi env:clientcollab-l1-cu operation_name:Microsoft.AspNetCore.server resource_name:"POST api/FileStore/Upload" ' + ' OR '.join(trace_strings)
    #driver.find_element(By.XPATH,'//*[@id="single-page-app_layout_page__main-content"]/div/div[1]/div[2]/div[2]/div/header/div/div/div/div/div[1]/div[1]/div/div/div[1]/div/div[1]/div/div/div/span').send_keys(result_string)
    driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[1]/div[1]/div/div/div/div[1]/div/div/div[1]/div/div[1]/div/div/div').send_keys(result_string)
    actions.send_keys(Keys.ENTER).perform()
    time.sleep(3)

    # To remove the existing string from 2nd Query box 
    driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[1]/div[1]/div/div/div/div[3]/div/div/div[1]/div/div[1]/div/div/button').click()
    
    # Steps to Insert Query in the second search
    trace_strings = [f'trace_id:{item} ' for item in list if item]
    result_string = 'env:clientcollab-l1-cu @http.url:"https://zuscu-d002-b139-pip-l1-axs01.centralus.cloudapp.azure.com/filescan/scan"  ' + ' OR '.join(trace_strings)
    driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/div/div[1]/div[1]/div/div/div/div[3]/div/div/div[1]/div/div[1]/div/div/div').send_keys(result_string)
    actions.send_keys(Keys.ENTER).perform()
    time.sleep(3)

    # Now Fetch the Table Data into the dataframe.
    pageSourceData = StringIO(driver.page_source)
    df_name = f"{value}"  # Create unique DataFrame name for each loop iteration
    traceDATA_DataframeTable_df1[df_name] = pd.read_html(pageSourceData)[0]  # Create a DataFrame and store it in the dictionary
    traceDATA_DataframeTable_df1[df_name].columns = ['Http.URL', 'Trace_ID', 'Duration']

#Writing data into Excel:
with pd.ExcelWriter('ResultedTraces.xlsx') as writer:
    # Write the first DataFrame to the first sheet
    traceDATA_DataframeTable.to_excel(writer, sheet_name= f'File Size = {fileSize[0]}')

    # Write the second DataFrame to the second sheet
    for i,value in enumerate(keyList[1:], start=1):
        try:
            traceDATA_DataframeTable_df1[keyList[i]].to_excel(writer, sheet_name=f'File Size = {fileSize[i]}')
        except:
            print(f'No Traces Available for File Size {keyList[i]}, However Trace ID is published in the downloaded CSV')
            dictName[keyList[i]].to_excel(writer, sheet_name=f'File Size = {fileSize[i]}')


1 20904490
2 14169117


KeyboardInterrupt: 

In [61]:
print(traceDATA_DataframeTable_df1)


{'20904490':                                             Http.URL  \
0  http://ltcollaborationapi.cchaxcess.com/ufmfil...   
1  http://ltcollaborationapi.cchaxcess.com/ufmfil...   
2  https://zuscultufmsan002.blob.core.windows.net...   
3  https://zuscu-d002-b139-pip-l1-axs01.centralus...   
4  https://ltcollaborationapi.cchaxcess.com/ufm/a...   
5  http://ltcollaborationapi.cchaxcess.com/api/Fi...   
6  https://ltcollaborationapi.cchaxcess.com/ufm/a...   
7  http://ltcollaborationapi.cchaxcess.com/api/Fi...   
8  http://taa-l1-cc-doculocker-service:8080/API/U...   
9  https://zuscultufmsan002.blob.core.windows.net...   

                           Trace_ID     Duration  
0  5f73c3c0df7c6e45e187b563e90300e2  13,364.6 ms  
1  b5f976bd823d2481cf606140c172e661     900.7 ms  
2  b5f976bd823d2481cf606140c172e661     323.4 ms  
3  b5f976bd823d2481cf606140c172e661     289.6 ms  
4  b5f976bd823d2481cf606140c172e661      53.7 ms  
5  b5f976bd823d2481cf606140c172e661      50.1 ms  
6  b5f976bd82

In [21]:
fileSize = [261888642,20904490,14169117,21348301]
for i,value in enumerate(fileSize[1:], start=1):
    df_name = f"_{value}"  # Create unique DataFrame name for each loop iteration
    dataframes[df_name] = pd.DataFrame(data)  # Create a DataFrame and store it in the dictionary

    print(i,value)

1 20904490
2 14169117
3 21348301


In [2]:
import json
import csv

# Load the JSON file
with open("C:\\Users\\Kapil.Sharma1\\Downloads\\ClientCollaborationLT1DashboardOnNewCluster--2024-11-18T09_15_56 (2).json", "r") as json_file:
    data = json.load(json_file)

# Open a CSV file to write
with open("dashboard.csv", "w", newline="") as csv_file:
    csv_writer = csv.writer(csv_file)

    # Write headers (customize based on the JSON structure)
    csv_writer.writerow(["Widget Title", "Query", "Type"])

    # Extract widget information from the JSON
    for widget in data.get("widgets", []):
        title = widget.get("definition", {}).get("title", "N/A")
        queries = widget.get("definition", {}).get("requests", [])
        query = "; ".join([q.get("query", "N/A") for q in queries])
        widget_type = widget.get("definition", {}).get("type", "N/A")

        # Write to CSV
        csv_writer.writerow([title, query, widget_type])
